In [2]:
import sklearn
import pyspark
from sklearn.metrics import average_precision_score
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
from matplotlib import pyplot as plt
from pathlib import Path
from sklearn.preprocessing import Normalizer
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.callbacks import TensorBoard
import datetime
from sklearn import metrics
import sklearn.preprocessing as preprocessing
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.layers import LayerNormalization
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm
from matplotlib import pyplot as plt
from pathlib import Path
from sklearn.preprocessing import Normalizer
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.callbacks import TensorBoard
import datetime
from sklearn import metrics
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from tensorflow import keras
from tensorflow.keras import regularizers




def pr_auc_score(y_true, y_pred_proba):
    ''' 
    Return the area under the Precision-Recall curve.
    
    Args:
        - y_true (pd.DataFrame): Dataframe with a unique identifier for each observation (first column) and the ground truth observations (second column).
        - y_pred_proba (pd.DataFrame): Dataframe with a unique identifier for each observation (first column) and the predicted probabilities estimates for the minority class (second column).
        
    Returns:
        float
    '''
    
    y_true_sorted = y_true.sort_values(by='ID').reset_index(drop=True)
    y_pred_proba_sorted = y_pred_proba.sort_values(by='ID').reset_index(drop=True)
    pr_auc_score = average_precision_score(np.ravel(y_true_sorted.iloc[:, 1]), np.ravel(y_pred_proba_sorted.iloc[:, 1]))

    return pr_auc_score









Y_train= pd.read_csv('Y_train.csv')
X_train = pd.read_csv('X_train.csv')
X_test= pd.read_csv('X_test.csv')



Y_train = Y_train['fraud_flag']



X_train=pd.get_dummies(X_train)
print(X_train)
X_test=pd.get_dummies(X_test)

df=X_train

missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending = False)
NAN_col = list(missing_values.to_dict().keys())
missing_values_df = pd.DataFrame(missing_values)
missing_values_df.reset_index(level=0, inplace=True)
missing_values_df.columns = ['Feature','Number of Missing Values']
missing_values_df['Percentage of Missing Values'] = (100.0*missing_values_df['Number of Missing Values'])/len(df)
print(missing_values_df)


          ID  cash_price1  cash_price2  cash_price3  cash_price4  cash_price5  \
0      85517        889.0          NaN          NaN          NaN          NaN   
1      51113        409.0          NaN          NaN          NaN          NaN   
2      83008       1399.0          NaN          NaN          NaN          NaN   
3      78712        689.0        119.0          NaN          NaN          NaN   
4      77846       1199.0          NaN          NaN          NaN          NaN   
...      ...          ...          ...          ...          ...          ...   
92785  21243        299.0          7.0          NaN          NaN          NaN   
92786  45891        898.0          NaN          NaN          NaN          NaN   
92787  42613        519.0        259.0        949.0          NaN          NaN   
92788  43567       1099.0       2099.0          NaN          NaN          NaN   
92789  68268        799.0          NaN          NaN          NaN          NaN   

       cash_price6  cash_pr

In [3]:
X_train=X_train.fillna(0)
X_test=X_test.fillna(0)
print(X_train)

          ID  cash_price1  cash_price2  cash_price3  cash_price4  cash_price5  \
0      85517        889.0          0.0          0.0          0.0          0.0   
1      51113        409.0          0.0          0.0          0.0          0.0   
2      83008       1399.0          0.0          0.0          0.0          0.0   
3      78712        689.0        119.0          0.0          0.0          0.0   
4      77846       1199.0          0.0          0.0          0.0          0.0   
...      ...          ...          ...          ...          ...          ...   
92785  21243        299.0          7.0          0.0          0.0          0.0   
92786  45891        898.0          0.0          0.0          0.0          0.0   
92787  42613        519.0        259.0        949.0          0.0          0.0   
92788  43567       1099.0       2099.0          0.0          0.0          0.0   
92789  68268        799.0          0.0          0.0          0.0          0.0   

       cash_price6  cash_pr

In [4]:
print(X_train.shape)
print(Y_train.shape)

(92790, 46128)
(92790,)


In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)





In [ ]:
print(X_train.shape)
print(Y_train.shape)

(92790, 146)

In [ ]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(272,)),
    keras.layers.Dense(50,kernel_regularizer=regularizers.L1L2(l1=0.001,l2=0.001),bias_regularizer=regularizers.L2(1e-4) ,activation='sigmoid'),
    #keras.layers.Dropout(rate=0.1, name='dropout'),
    keras.layers.Dense(30,kernel_regularizer=regularizers.L1L2(l1=0.001,l2=0.001),bias_regularizer=regularizers.L2(1e-4) , activation='tanh'),
    keras.layers.Dense(1,activation='sigmoid'),
    ])



tensorboard_callback = TensorBoard(log_dir="./call")
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=["mse"],
)

model.fit(X_train, Y_train, epochs=200, batch_size=1,
          callbacks=[tensorboard_callback])

MemoryError: Unable to allocate 31.9 GiB for an array with shape (46128, 92790) and data type object